In [ ]:
# hana_connection.py
from hdbcli import dbapi
import ssl
import os
from dotenv import load_dotenv  
load_dotenv()  # Load environment variables from .env file

class HanaConfig:
    HOST = "your-hana-host"        # e.g. "hana.mycorp.internal"
    PORT = 30015                   # e.g. 3NN15 on-prem / tenant port [web:130][web:132]
    USER = os.getenv("HANA_USER")
    PASSWORD = os.getenv("HANA_PASSWORD")
    PASSWORD = os
    # For HANA Cloud use encrypt=True, sslValidateCertificate=True [web:131]
    ENCRYPT = True
    SSL_VALIDATE_CERT = False      # adjust to your environment
    SSL_KEYSTORE = None            # path to trust store if needed

def create_hana_connection():
    """
    Create and return a HANA dbapi connection for use inside MCP tools.
    Caller is responsible for closing it.
    """
    conn_kwargs = dict(
        address=HanaConfig.HOST,
        port=HanaConfig.PORT,
        user=HanaConfig.USER,
        password=HanaConfig.PASSWORD,
    )

    # Optional: TLS options (mainly for HANA Cloud) [web:131][web:143]
    conn_kwargs.update(
        encrypt=HanaConfig.ENCRYPT,
        sslValidateCertificate=HanaConfig.SSL_VALIDATE_CERT,
    )
    if HanaConfig.SSL_KEYSTORE:
        conn_kwargs["sslCryptoProvider"] = "openssl"
        conn_kwargs["sslTrustStore"] = HanaConfig.SSL_KEYSTORE

    conn = dbapi.connect(**conn_kwargs)
    return conn
# inside your MCP server tool handler
from hana_connection import create_hana_connection

def run_sql(sql: str, params=None):
    conn = create_hana_connection()
    try:
        cursor = conn.cursor()
        cursor.execute(sql, params or [])
        rows = cursor.fetchall()
        columns = [c[0] for c in cursor.description]
        return {"columns": columns, "rows": rows}
    finally:
        conn.close()
